# 第4章 モデル作成
### 4.1.2 モデル作成の流れ

In [63]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

cancer=load_breast_cancer()
X=cancer.data
X

array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
        1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
        8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
        8.758e-02],
       ...,
       [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
        7.820e-02],
       [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
        1.240e-01],
       [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
        7.039e-02]])

In [64]:
y=cancer.target
import pandas as pd
df_x=pd.DataFrame(X)
df_y=pd.DataFrame(y)
df_x

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


In [65]:
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
import numpy as np

scores=[]
kf=KFold(n_splits=4,shuffle=True,random_state=71)
for tr_idx,va_idx in (kf.split(df_x)):
    tr_x,va_x=df_x.iloc[tr_idx],df_x.iloc[va_idx]
    tr_y,va_y=df_y.iloc[tr_idx],df_y.iloc[va_idx]
    
    model=DecisionTreeClassifier()
    model.fit(tr_x,tr_y)
    va_pred=model.predict(va_x)
    score=log_loss(va_y,va_pred)
    scores.append(score)

print(np.mean(scores))

2.2473576315972914


### 4.3.4 GBDTの実装

In [66]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y)

In [67]:
import xgboost as xgb
from sklearn.metrics import log_loss

#特徴量と目的変数をxgboostのデータ構造に変換する
dtrain=xgb.DMatrix(X_train,label=y_train)
dvalid=xgb.DMatrix(X_test,label=y_test)

#ハイパーパラメータの設定
params={'objective':'binary:logistic','silent':1,'randomstate':71}
num_round=20

#学習の実行
#バリデーションデータもモデルに渡し、学習の進行とともにスコアがどう変わるかモニタリングする
#watchlistには学習データおよびバリデーションデータをセットする
watchlist=[(dtrain,'train'),(dvalid,'eval')]
model=xgb.train(params,dtrain,num_round,evals=watchlist)

#バリデーションデータでのスコアの確認
va_pred=model.predict(dvalid)
score=log_loss(y_test,va_pred)
print('logloss:{}'.format(score))

[19:42:48] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { randomstate, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-error:0.01409	eval-error:0.02797
[1]	train-error:0.00704	eval-error:0.02098
[2]	train-error:0.00704	eval-error:0.02797
[3]	train-error:0.00470	eval-error:0.02098
[4]	train-error:0.00235	eval-error:0.02098
[5]	train-error:0.00235	eval-error:0.02797
[6]	train-error:0.00235	eval-error:0.02797
[7]	train-error:0.00470	eval-error:0.02098
[8]	train-error:0.00235	eval-error:0.02098
[9]	train-error:0.00235	eval-error:0.02098
[10]	train-error:0.00235	eval-error:0.02098
[11]	train-error:0.00235	eval-error:0.02098
[12]	train-error:0.00235	eval-error:0.02098
[13]	train-error:0.00235	eval-error:0.020

In [54]:
import xgboost as xgb
from sklearn.metrics import log_loss

#特徴量と目的変数をxgboostのデータ構造に変換する
dtrain=xgb.DMatrix(X_train,label=y_train)
dvalid=xgb.DMatrix(X_test,label=y_test)
dtest=xgb.DMatrix(X,label=y)

#ハイパーパラメータの設定
params={'objective':'binary:logistic','silent':1,'randomstate':71,'eval_metric':'logloss'}
num_round=500

#学習の実行
#バリデーションデータもモデルに渡し、学習の進行とともにスコアがどう変わるかモニタリングする
#watchlistには学習データおよびバリデーションデータをセットする
watchlist=[(dtrain,'train'),(dvalid,'eval')]
model=xgb.train(params,dtrain,num_round,evals=watchlist,early_stopping_rounds=5) #early_stopping_rounds分だけ様子を見る

#バリデーションデータでのスコアの確認
va_pred=model.predict(dvalid)
score=log_loss(y_test,va_pred)
print('logloss:{}'.format(score))

[00:13:51] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { randomstate, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-logloss:0.46486	eval-logloss:0.48718
Multiple eval metrics have been passed: 'eval-logloss' will be used for early stopping.

Will train until eval-logloss hasn't improved in 5 rounds.
[1]	train-logloss:0.33325	eval-logloss:0.37689
[2]	train-logloss:0.24881	eval-logloss:0.30256
[3]	train-logloss:0.19045	eval-logloss:0.24353
[4]	train-logloss:0.14932	eval-logloss:0.20883
[5]	train-logloss:0.12070	eval-logloss:0.18507
[6]	train-logloss:0.09843	eval-logloss:0.15989
[7]	train-logloss:0.08184	eval-logloss:0.14628
[8]	train-logloss:0.06934	eval-logloss:0.13186
[9]	train-logloss:0.05997	eval-l

In [55]:
#予測するときは、early stoppingを行った場合n_tree_limitパラメータを設定する
pred=model.predict(dtest,ntree_limit=model.best_ntree_limit)

In [61]:
import lightgbm as lgb
from sklearn.metrics import log_loss

#特徴量と目的変数をlgbのデータ構造に変換する
lgb_train=lgb.Dataset(X_train,y_train)
lgb_eval=lgb.Dataset(X_test,y_test)

#ハイパーパラメータの設定
params={'objective':'binary','seed':71,'vervose':0,'metrics':'binary_logloss'}
num_round=100

#学習の実行
#カテゴリ変数をパラメータで指定している
#categorical_features={'product','medical_info_b2','medical_info_b3'}
watchlist=[(dtrain,'train'),(dvalid,'eval')]
model=lgb.train(params,lgb_train,num_boost_round=num_round,
#                categorical_feature=categorical_features,
                valid_names=['train','valid'],valid_sets=[lgb_train,lgb_eval])

#バリデーションデータでのスコアの確認
va_pred=model.predict(X_test)
score=log_loss(y_test,va_pred)
print('logloss:{}'.format(score))

[1]	train's binary_logloss: 0.570148	valid's binary_logloss: 0.60288
[2]	train's binary_logloss: 0.504462	valid's binary_logloss: 0.540086
[3]	train's binary_logloss: 0.452677	valid's binary_logloss: 0.487231
[4]	train's binary_logloss: 0.409785	valid's binary_logloss: 0.441593
[5]	train's binary_logloss: 0.373453	valid's binary_logloss: 0.402942
[6]	train's binary_logloss: 0.338981	valid's binary_logloss: 0.372401
[7]	train's binary_logloss: 0.311156	valid's binary_logloss: 0.342663
[8]	train's binary_logloss: 0.2851	valid's binary_logloss: 0.31745
[9]	train's binary_logloss: 0.2615	valid's binary_logloss: 0.297295
[10]	train's binary_logloss: 0.242534	valid's binary_logloss: 0.277161
[11]	train's binary_logloss: 0.224498	valid's binary_logloss: 0.261777
[12]	train's binary_logloss: 0.207652	valid's binary_logloss: 0.247412
[13]	train's binary_logloss: 0.194299	valid's binary_logloss: 0.232947
[14]	train's binary_logloss: 0.180114	valid's binary_logloss: 0.221554
[15]	train's binary_l

### 4.4.4 ニューラルネットの実装

In [70]:
!pip install tensorflow

  Created wheel for absl-py: filename=absl_py-0.9.0-py3-none-any.whl size=121935 sha256=b8cf57d1f135112ac959fe5d285e578091aec07c59d967b5e0498d21b953ef8f
  Stored in directory: c:\users\nakam\appdata\local\pip\cache\wheels\cc\af\1a\498a24d0730ef484019e007bb9e8cef3ac00311a672c049a3e
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4835 sha256=4e8636ea4f493275348ddeeff63c814f089b64641b4e064e33311f615d401799
  Stored in directory: c:\users\nakam\appdata\local\pip\cache\wheels\3f\e3\ec\8a8336ff196023622fbcb36de0c5a5c218cbb24111d1d4c7f2
Successfully built absl-py termcolor


In [74]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

cancer=load_breast_cancer()
X=cancer.data
X

array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
        1.189e-01],
       [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
        8.902e-02],
       [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
        8.758e-02],
       ...,
       [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
        7.820e-02],
       [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
        1.240e-01],
       [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
        7.039e-02]])

In [75]:
from sklearn.model_selection import train_test_split
tr_x,test_x,tr_y,test_y=train_test_split(X,y)
tr_x,va_x,tr_y,va_y=train_test_split(tr_x,tr_y)

In [88]:
from keras.layers import Dense, Dropout
from keras.models import Sequential
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler

#データのスケーリング
scaler=StandardScaler()
tr_x=scaler.fit_transform(tr_x)
va_x=scaler.transform(va_x)
test_x=scaler.transform(test_x)

#ニューラルネットモデルの構築
model=Sequential()
model.add(Dense(256,activation='relu',input_shape=(tr_x.shape[1],))) #1層目が256個のノードからなる
model.add(Dropout(0.2))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',
             optimizer='adam',metrics=['accuracy'])

#学習の実行
#validationデータもモデルに渡し、学習の進行とともにスコアがどう変わるかモニタリングする
batch_size=128
epochs=10
history=model.fit(tr_x,tr_y,
                 batch_size=batch_size,epochs=epochs,
                 verbose=1,validation_data=(va_x,va_y))

#validationデータでのスコアの確認
va_pred=model.predict(va_x)
score=log_loss(va_y,va_pred,eps=1e-7)
print(score)

Epoch 1/10
3/3 [==============================] - 0s 158ms/step - loss: 0.5739 - accuracy: 0.7900 - val_loss: 0.3420 - val_accuracy: 0.9252
Epoch 2/10
3/3 [==============================] - 0s 46ms/step - loss: 0.3231 - accuracy: 0.9530 - val_loss: 0.2216 - val_accuracy: 0.9346
Epoch 3/10
3/3 [==============================] - 0s 40ms/step - loss: 0.2030 - accuracy: 0.9561 - val_loss: 0.1692 - val_accuracy: 0.9439
Epoch 4/10
3/3 [==============================] - 0s 41ms/step - loss: 0.1474 - accuracy: 0.9624 - val_loss: 0.1462 - val_accuracy: 0.9439
Epoch 5/10
3/3 [==============================] - 0s 32ms/step - loss: 0.1210 - accuracy: 0.9592 - val_loss: 0.1322 - val_accuracy: 0.9439
Epoch 6/10
3/3 [==============================] - 0s 37ms/step - loss: 0.1029 - accuracy: 0.9624 - val_loss: 0.1211 - val_accuracy: 0.9439
Epoch 7/10
3/3 [==============================] - 0s 34ms/step - loss: 0.0859 - accuracy: 0.9749 - val_loss: 0.1143 - val_accuracy: 0.9626
Epoch 8/10
3/3 [==========

In [91]:
from keras.callbacks import EarlyStopping

#アーリーストッピングの観察するroundを20とする
#restore_best_weightsを設定することで、最適なエポックのモデルを使用する
epochs=50
early_stopping=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

history=model.fit(tr_x,tr_y,
                 batch_size=batch_size,epochs=epochs,
                 verbose=1,validation_data=(va_x,va_y),callbacks=[early_stopping])

Epoch 1/50
3/3 [==============================] - 0s 29ms/step - loss: 0.0530 - accuracy: 0.9812 - val_loss: 0.1046 - val_accuracy: 0.9813
Epoch 2/50
3/3 [==============================] - 0s 34ms/step - loss: 0.0478 - accuracy: 0.9843 - val_loss: 0.1050 - val_accuracy: 0.9813
Epoch 3/50
3/3 [==============================] - 0s 35ms/step - loss: 0.0449 - accuracy: 0.9875 - val_loss: 0.1049 - val_accuracy: 0.9813
Epoch 4/50
3/3 [==============================] - 0s 35ms/step - loss: 0.0420 - accuracy: 0.9875 - val_loss: 0.1045 - val_accuracy: 0.9813
Epoch 5/50
3/3 [==============================] - 0s 32ms/step - loss: 0.0377 - accuracy: 0.9843 - val_loss: 0.1044 - val_accuracy: 0.9813
Epoch 6/50
3/3 [==============================] - 0s 30ms/step - loss: 0.0376 - accuracy: 0.9906 - val_loss: 0.1044 - val_accuracy: 0.9813
Epoch 7/50
3/3 [==============================] - 0s 35ms/step - loss: 0.0355 - accuracy: 0.9906 - val_loss: 0.1043 - val_accuracy: 0.9813
Epoch 8/50
3/3 [===========

### 4.5.4 線形モデルの実装

In [93]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler

#データのスケーリング
scaler=StandardScaler()
tr_x=scaler.fit_transform(tr_x)
va_x=scaler.transform(va_x)
test_x=scaler.transform(test_x)

#線形モデルの構築・学習
model=LogisticRegression(C=1.0)
model.fit(tr_x,tr_y)

#validation_dataでのスコアの確認
va_pred=model.predict_proba(va_x)
score=log_loss(va_y,va_pred)
print(score)

0.11196744951332009
